## Openimagesv7_재구현

In [ ]:
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0) if torch.cuda.is_available() else "NO_GPU"

(True, 'NVIDIA GeForce RTX 5060')

In [ ]:
import os
from PIL import Image
import torch
import pandas as pd
import numpy as np
from gem import create_gem_model, get_gem_img_transform
from tqdm import tqdm

# =============================
# torch.load wrapper
# =============================
_torch_load = torch.load
def torch_load_wrapper(*args, **kwargs):
    kwargs["weights_only"] = False
    return _torch_load(*args, **kwargs)
torch.load = torch_load_wrapper

device = "cuda" if torch.cuda.is_available() else "cpu"

C:\Users\Konyang\miniconda3\envs\clip\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Konyang\miniconda3\envs\clip\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:
model_name = "ViT-B/16"
pretrained = "laion400m_e32"
gem_model = create_gem_model(model_name=model_name, pretrained=pretrained, device=device)
gem_model.eval()
gem_img_transform = get_gem_img_transform()

100%|███████████████████████████████████████| 599M/599M [00:20<00:00, 29.8MiB/s]


In [ ]:
image_root = r"C:\Users\Konyang\Desktop\open-vocab 데이터\openimagesv7\data"

class_csv = r"C:\Users\Konyang\Desktop\open-vocab 데이터\openimagesv7\oidv7-class-descriptions-points (1).csv"
anno_csv  = r"C:\Users\Konyang\Desktop\open-vocab 데이터\openimagesv7\oidv7-val-annotations-point-labels.csv"

class_df = pd.read_csv(class_csv)
anno_df_raw = pd.read_csv(anno_csv)

anno_df = anno_df_raw.rename(columns={
    "ImageId": "ImageID",
    "Label": "LabelName",
    "X": "X",
    "Y": "Y"
})

label_to_name = dict(zip(class_df["LabelName"], class_df["DisplayName"]))

In [ ]:
def compute_point_miou_for_image(logits, points_df, image_w, image_h, class_list):

    logits = logits[0]   # [C, H, W]
    C, H_out, W_out = logits.shape

    pred_masks = {}
    for idx, cls in enumerate(class_list):
        prob = torch.sigmoid(logits[idx])
        pred_masks[cls] = (prob > 0.5).cpu().numpy()

    intersection = np.zeros(C)
    union = np.zeros(C)

    class_to_idx = {cls: i for i, cls in enumerate(class_list)}

    for _, row in points_df.iterrows():

        label_id = row["LabelName"]
        if label_id not in label_to_name:
            continue

        gt_class = label_to_name[label_id]
        if gt_class not in class_to_idx:
            continue

        gx = int((row["X"] / image_w) * W_out)
        gy = int((row["Y"] / image_h) * H_out)

        gx = max(0, min(W_out - 1, gx))
        gy = max(0, min(H_out - 1, gy))

        for cls, idx in class_to_idx.items():
            pred_pos = pred_masks[cls][gy, gx]
            gt_pos = (cls == gt_class)

            if gt_pos or pred_pos:
                union[idx] += 1
            if gt_pos and pred_pos:
                intersection[idx] += 1

    valid = union > 0
    if valid.sum() == 0:
        return 0.0, intersection, union, class_to_idx

    miou = (intersection[valid] / (union[valid] + 1e-6)).mean()
    return miou, intersection, union, class_to_idx



In [ ]:
all_image_ids = anno_df["ImageID"].unique()
total_images = len(all_image_ids)
print(f"총 평가 이미지 수: {total_images}")

CHUNK_SIZE = 500
SAVE_PATH = "pmIoU_results_openclip.npy"

if os.path.exists(SAVE_PATH):
    miou_results = list(np.load(SAVE_PATH, allow_pickle=True))
    start_index = len(miou_results)
    print(f"기존 진행 발견 → {start_index}개 완료됨. 이어서 진행.")
else:
    miou_results = []
    start_index = 0

# ★ 클래스별 누적 통계
class_intersection = {}
class_union = {}

총 평가 이미지 수: 36702


In [ ]:
for start in range(start_index, total_images, CHUNK_SIZE):

    end = min(start + CHUNK_SIZE, total_images)
    chunk_ids = all_image_ids[start:end]

    print(f"\n=== [{start} ~ {end}] {len(chunk_ids)}개 이미지 평가중... ===")

    for image_id in tqdm(chunk_ids, desc=f"Chunk {start}-{end}"):

        img_path = os.path.join(image_root, image_id + ".jpg")

        if not os.path.exists(img_path):
            miou_results.append(None)
            continue

        point_rows = anno_df[anno_df["ImageID"] == image_id]
        gt_label_ids = point_rows["LabelName"].unique()

        class_list = [label_to_name[l] for l in gt_label_ids if l in label_to_name]
        if len(class_list) == 0:
            miou_results.append(None)
            continue

        img_pil = Image.open(img_path).convert("RGB")
        W, H = img_pil.size
        img_tensor = gem_img_transform(img_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            logits = gem_model(img_tensor, class_list)

        miou, inter, union, class_to_idx = compute_point_miou_for_image(
            logits, point_rows, W, H, class_list
        )

        miou_results.append(miou)

        # 클래스별 통계 업데이트
        for cls, idx in class_to_idx.items():

            if cls not in class_intersection:
                class_intersection[cls] = 0.0
                class_union[cls] = 0.0

            class_intersection[cls] += inter[idx]
            class_union[cls] += union[idx]

    # 청크 저장
    np.save(SAVE_PATH, np.array(miou_results, dtype=object))
    print(f"✓ 청크 저장 완료 ({len(miou_results)}개 완료됨)")


# ---------------------------
# 최종 결과
# ---------------------------
valid_scores = [m for m in miou_results if m is not None]
final_miou = np.mean(valid_scores)

print("\n====================================")
print(f" Final Dataset p-mIoU = {final_miou*100:.2f}")
print("====================================")

print("\n========== 클래스별 mIoU ==========")
for cls in sorted(class_intersection.keys()):
    if class_union[cls] > 0:
        score = class_intersection[cls] / (class_union[cls] + 1e-6)
        print(f"{cls:30s} : {score*100:.2f}")


=== [0 ~ 500] 500개 이미지 평가중... ===


Chunk 0-500: 100%|███████████████████████████████████████████████████████████████████| 500/500 [01:36<00:00,  5.17it/s]


✓ 청크 저장 완료 (500개 완료됨)

=== [500 ~ 1000] 500개 이미지 평가중... ===


Chunk 500-1000: 100%|████████████████████████████████████████████████████████████████| 500/500 [01:37<00:00,  5.11it/s]


✓ 청크 저장 완료 (1000개 완료됨)

=== [1000 ~ 1500] 500개 이미지 평가중... ===


...

Chunk 36000-36500: 100%|██████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 12599.75it/s]


✓ 청크 저장 완료 (36500개 완료됨)

=== [36500 ~ 36702] 202개 이미지 평가중... ===


Chunk 36500-36702: 100%|██████████████████████████████████████████████████████████| 202/202 [00:00<00:00, 20004.00it/s]

✓ 청크 저장 완료 (36702개 완료됨)



 Final Dataset p-mIoU = 47.44

========== 클래스별 mIoU ==========
Abdomen                        : 4.17
Accordion                      : 25.35
Acrylic paint                  : 25.00
Action figure                  : 32.35
Adapter (Electrical)           : 11.72
Adhesive                       : 11.94
Adhesive tape                  : 22.58
Adult                          : 21.43
Advertising                    : 10.62
Agate                          : 7.14
Agave                          : 51.35
Agriculture                    : 20.97
Airboat                        : 35.14
Aircraft                       : 20.48
Aircraft carrier               : 1.92
Airline                        : 18.18
Airplane                       : 14.88
Airport                        : 17.39
Airship                        : 31.34
Alarm clock                    : 39.33
Alcoholic beverage             : 17.11
Algae                          : 41.67
All-terrain vehicle            : 6.25
Alligator                      : 43.18
Almo